In [42]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import glob

from keras.utils.np_utils import to_categorical
import keras
from keras.models import Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Input
#from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, log_loss, accuracy_score

In [60]:
x_train = pd.read_csv('csvTrainImages 13440x1024.csv').to_numpy()
y_train = pd.read_csv('csvTrainLabel 13440x1.csv').to_numpy()-1
x_test = pd.read_csv('csvTestImages 3360x1024.csv').to_numpy()
y_test = pd.read_csv('csvTestLabel 3360x1.csv').to_numpy()-1

In [61]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(13439, 1024)
(13439, 1)
(3359, 1024)
(3359, 1)


In [62]:
x_train = x_train.reshape(-1,32,32,1)
x_test = x_test.reshape(-1,32,32,1)

x_train = x_train / 255
x_test = x_test / 255

print(x_train.shape, x_test.shape)


# array = np.random.choice(50,25)

# plt.figure(figsize=(10,10))

# for i in range(25):
#     plt.subplot(5,5,i+1)
#     plt.xticks([])
#     plt.yticks([])
#     plt.imshow(x_train[array[i]],cmap = 'gray')
# #     j+=1
# plt.show()
    

(13439, 32, 32, 1) (3359, 32, 32, 1)


In [63]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [64]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, stratify = y_train)


In [65]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(10751, 32, 32, 1)
(10751, 28)
(2688, 32, 32, 1)
(2688, 28)


In [66]:
def get_model():
    In = Input(shape=(32,32,1))
    x = Conv2D(32, (5,5), padding="same", activation="relu")(In)
    x = Conv2D(32, (5,5), activation="relu")(x)
    x = Conv2D(32, (5,5), activation="relu")(x)
    x = MaxPooling2D((2,2))(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(64, (5,5), padding="same", activation="relu")(x)
    x = Conv2D(64, (5,5), activation="relu")(x)
    x = Conv2D(64, (5,5), activation="relu")(x)
    x = MaxPooling2D((2,2))(x)
    x = BatchNormalization()(x)
    
    x = Flatten()(x)
    x = Dense(128, activation="relu")(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.4)(x)
    
    Out = Dense(28, activation="softmax")(x)
    
    model = Model(In, Out)
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

model = get_model()
#keras.utils.vis_utils.plot_model(model, show_shapes=True, show_layer_names=True)

In [67]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # 

In [68]:
batch_size = 64
epochs = 50

train_gen = datagen.flow(x_train, y_train, batch_size=batch_size)
test_gen = datagen.flow(x_test, y_test, batch_size=batch_size)

In [69]:
model_checkpoint_callback = ModelCheckpoint(
    filepath="best.hdf5",
    monitor='val_accuracy', 
    verbose=1, 
    save_best_only=True, 
    mode='max')


history = model.fit(train_gen, 
                              epochs = epochs,
                              verbose = 0,
                              steps_per_epoch = x_train.shape[0] // batch_size,
                              validation_data = test_gen,
                              validation_steps = x_test.shape[0] // batch_size,
                              callbacks=[model_checkpoint_callback])


Epoch 1: val_accuracy improved from -inf to 0.04898, saving model to best.hdf5

Epoch 2: val_accuracy improved from 0.04898 to 0.12500, saving model to best.hdf5

Epoch 3: val_accuracy improved from 0.12500 to 0.50361, saving model to best.hdf5

Epoch 4: val_accuracy improved from 0.50361 to 0.74279, saving model to best.hdf5

Epoch 5: val_accuracy improved from 0.74279 to 0.84495, saving model to best.hdf5

Epoch 6: val_accuracy did not improve from 0.84495

Epoch 7: val_accuracy improved from 0.84495 to 0.91647, saving model to best.hdf5

Epoch 8: val_accuracy did not improve from 0.91647

Epoch 9: val_accuracy improved from 0.91647 to 0.92909, saving model to best.hdf5

Epoch 10: val_accuracy did not improve from 0.92909

Epoch 11: val_accuracy improved from 0.92909 to 0.93419, saving model to best.hdf5

Epoch 12: val_accuracy did not improve from 0.93419

Epoch 13: val_accuracy did not improve from 0.93419

Epoch 14: val_accuracy improved from 0.93419 to 0.94862, saving model to b

In [70]:
model.load_weights = ('best.hdf5')
print('training accuracy{} '.format(model.evaluate(x_train,y_train)))
print('validation accuracy {}'.format(model.evaluate(x_val,y_val)))

336/336 [==============================] - 27s 79ms/step - loss: 0.0496 - accuracy: 0.9861
training accuracy[0.04957479238510132, 0.9861408472061157] 
84/84 [==============================] - 7s 79ms/step - loss: 0.1545 - accuracy: 0.9669
validation accuracy [0.15448209643363953, 0.9668898582458496]


In [71]:
y_pred = model.predict(x_val)
pred = np.argmax(y_pred, axis=1) + 1 
ground = np.argmax(y_val, axis=1) + 1

print(classification_report(ground,pred))

              precision    recall  f1-score   support

           1       0.99      0.96      0.97        96
           2       0.99      1.00      0.99        96
           3       0.90      0.98      0.94        96
           4       0.99      0.91      0.95        96
           5       0.97      0.98      0.97        96
           6       0.93      0.98      0.95        96
           7       0.99      0.95      0.97        96
           8       0.88      0.98      0.93        96
           9       0.98      0.85      0.91        96
          10       0.90      0.97      0.93        96
          11       0.95      0.95      0.95        96
          12       0.99      1.00      0.99        96
          13       1.00      0.99      0.99        96
          14       1.00      0.98      0.99        96
          15       0.99      1.00      0.99        96
          16       0.96      0.98      0.97        96
          17       0.99      0.96      0.97        96
          18       0.95    

In [72]:
print('validation accuracy {}'.format(model.evaluate(x_test,y_test)))

105/105 [==============================] - 8s 80ms/step - loss: 0.1627 - accuracy: 0.9670
validation accuracy [0.16266126930713654, 0.9669544696807861]


In [73]:
y_pred = model.predict(x_test)
pred = np.argmax(y_pred, axis=1) + 1 
ground = np.argmax(y_test, axis=1) + 1

print(classification_report(ground,pred))

              precision    recall  f1-score   support

           1       1.00      0.94      0.97       119
           2       0.96      1.00      0.98       120
           3       0.93      0.95      0.94       120
           4       0.99      0.95      0.97       120
           5       0.98      0.97      0.98       120
           6       0.91      0.99      0.95       120
           7       0.97      0.95      0.96       120
           8       0.91      0.97      0.94       120
           9       0.98      0.88      0.93       120
          10       0.87      0.98      0.93       120
          11       0.96      0.92      0.94       120
          12       0.98      0.99      0.99       120
          13       0.99      0.99      0.99       120
          14       0.96      0.98      0.97       120
          15       1.00      0.96      0.98       120
          16       0.95      0.99      0.97       120
          17       0.98      0.95      0.97       120
          18       0.98    